In [1]:
!pip3 install newspaper3k
!pip3 install feedparser

  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13537 sha256=c6ea85fbbf121bc376a5c414c32116339b984109da409959b5952e1f8cfae5ab
  Stored in directory: c:\users\sheel\appdata\local\pip\cache\wheels\99\74\83\8fac1c8d9c648cfabebbbffe97a889f6624817f3aa0bbe6c09
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3354 sha256=92aef791b9eb723af9334e02cd18e796b9aca4d9439eb8a4a4a860cb5d5a0ec9
  Stored in directory: c:\users\sheel\appdata\local\pip\cache\wheels\b6\09\68\a9f15498ac02c23dde29f18745bc6a6f574ba4ab41861a3575
  Created wheel for jieba3k: filename=jieba3k-0.35.1-py3-none-any.whl size=7398406 sha256=0b599fc6a9327bbb85afa793d230a98b2c467163f6cd5f4af03c4b118171e1ae
  Stored in directory: c:\users\sheel\appdata\local\pip\cache\wheels\1f\7e\0c\54f3b0f5164278677899f2db08f2b07943ce2d024a3c862afb
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6065 sha256=f677783ec342646d653bc01a48b0d50b352bb06b123b0aa

In [ ]:
import os
from google.colab import drive

# Mount google drive
DRIVE_MOUNT='/content/gdrive'
drive.mount(DRIVE_MOUNT)

# create folder to write data to
Fake_News=os.path.join(DRIVE_MOUNT, 'My Drive', 'Fake News')
Project=os.path.join(Fake_News, 'Project')
os.makedirs(Project, exist_ok=True)

Mounted at /content/gdrive


In [9]:
import json
dictionary = {
  "breitbart": {
    "link": "https://www.breitbart.com/search/?s=crypto#gsc.tab=0&gsc.q=crypto&gsc.page=1",
    "rss" : "https://www.breitbart.com/search/?s=crypto"
  }
}

In [10]:
json_object = json.dumps(dictionary, indent = 4)
with open("NewsPapers.json", "w") as outfile: 
    outfile.write(json_object)

In [11]:
import feedparser as fp
import json
import newspaper
from newspaper import Article
from time import mktime
from datetime import datetime

# Set the limit for number of articles to download
LIMIT = 14500

data = {}
data['newspapers'] = {}

# Loads the JSON files with news sites
with open('NewsPapers.json') as data_file:
    companies = json.load(data_file)

count = 1

# Iterate through each news company
for company, value in companies.items():
    # If a RSS link is provided in the JSON file, this will be the first choice.
    # Reason for this is that, RSS feeds often give more consistent and correct data.
    # If you do not want to scrape from the RSS-feed, just leave the RSS attr empty in the JSON file.
    if 'rss' in value:
        d = fp.parse(value['rss'])
        print("Downloading articles from ", company)
        newsPaper = {
            "rss": value['rss'],
            "link": value['link'],
            "articles": []
        }
        for entry in d.entries:
            # Check if publish date is provided, if no the article is skipped.
            # This is done to keep consistency in the data and to keep the script from crashing.
            if hasattr(entry, 'published'):
                if count > LIMIT:
                    break
                article = {}
                article['link'] = entry.link
                date = entry.published_parsed
                article['published'] = datetime.fromtimestamp(mktime(date)).isoformat()
                try:
                    content = Article(entry.link)
                    content.download()
                    content.parse()
                except Exception as e:
                    # If the download for some reason fails (ex. 404) the script will continue downloading
                    # the next article.
                    print(e)
                    print("continuing...")
                    continue
                article['title'] = content.title
                article['text'] = content.text
                newsPaper['articles'].append(article)
                print(count, "articles downloaded from", company, ", url: ", entry.link)
                count = count + 1
    else:
        # This is the fallback method if a RSS-feed link is not provided.
        # It uses the python newspaper library to extract articles
        print("Building site for ", company)
        paper = newspaper.build(value['link'], memoize_articles=False)
        newsPaper = {
            "link": value['link'],
            "articles": []
        }
        noneTypeCount = 0
        for content in paper.articles:
            if count > LIMIT:
                break
            try:
                content.download()
                content.parse()
            except Exception as e:
                print(e)
                print("continuing...")
                continue
            # Again, for consistency, if there is no found publish date the article will be skipped.
            # After 10 downloaded articles from the same newspaper without publish date, the company will be skipped.
            if content.publish_date is None:
                print(count, " Article has date of type None...")
                noneTypeCount = noneTypeCount + 1
                if noneTypeCount > 100:
                    print("Too many noneType dates, aborting...")
                    noneTypeCount = 0
                    break
                count = count + 1
                continue
            article = {}
            article['title'] = content.title
            article['text'] = content.text
            article['link'] = content.url
            article['published'] = content.publish_date.isoformat()
            newsPaper['articles'].append(article)
            print(count, "articles downloaded from", company, " using newspaper, url: ", content.url)
            count = count + 1
            noneTypeCount = 0
    count = 1
    data['newspapers'][company] = newsPaper

# Finally it saves the articles as a JSON-file.
try:
    with open('scraped_articles.json', 'w') as outfile:
        json.dump(data, outfile)
except Exception as e: print(e)

In [12]:
with open('scraped_articles.json') as json_data:
    d = json.load(json_data)

In [13]:
for i, site in enumerate((list(d['newspapers']))):
    print(i, site)

0 breitbart


In [14]:

import pandas as pd
for i, site in enumerate((list(d['newspapers']))):
    articles = list(d['newspapers'][site]['articles'])
    if i == 0:
        df = pd.DataFrame.from_dict(articles)
        df["site"] = site
    else:
        new_df = pd.DataFrame.from_dict(articles)
        new_df["site"] = site
        df = pd.concat([df, new_df], ignore_index = True)

In [15]:
df.shape

(0, 1)

In [16]:
df.head()

,site


In [ ]:
!cp scraped_articles.csv "/content/gdrive/My Drive/Fake News/Project/"

In [ ]:
df.to_csv('scraped_articles.csv')
